# Portfolio Statistics Aggregator

This notebook lets you input the summary statistics for each period (e.g. each 7‑day period) and then computes an aggregated overall statistic for the full combined period. The input statistics for each period are assumed to be:

- **Final Portfolio Value** (as a growth factor over the period, e.g. 1.0179 means a 1.79% gain)
- **Annualized Sharpe Ratio** (calculated from the 2‑minute returns within that period using the formula: 
  ```python
  ann_sharpe = (mean_ret / std_ret) * np.sqrt(195 * 252)
  ```
  where 195 is the number of intervals per day and 252 is the number of trading days per year)
- **Maximum Drawdown** (given as a negative percentage, e.g. -7.39%)
- **Win Rate** (the percentage of intervals with positive returns)

### Aggregation Method

- **Final Portfolio Value:** The overall growth factor is computed as the product of each period's final portfolio value.
- **Annualized Sharpe Ratio:** As an approximation, the overall Sharpe is taken as the simple average of the period Sharpe ratios (this is only an estimate since Sharpe ratios are not directly additive).
- **Maximum Drawdown:** We take the worst (most negative) drawdown across the periods as the combined drawdown.
- **Win Rate:** With equal-length periods, the overall win rate is approximated by the average of the period win rates.

Feel free to adjust these methods if you have additional information (such as the number of intervals per period) or if you have the underlying return series available.

In [4]:
import numpy as np
import pandas as pd

# --- Input: Summary statistics for each period ---
data = {
    'Period': ['Period 1', 'Period 2', 'Period 3', 'Period 4'],
    'Final Portfolio Value': [1.0403, 1.0520, 1.0326, 1.0196],
    'Annualized Sharpe': [3.8151, 4.6947, 3.1946, 2.1642],
    'Max Drawdown': [-6.77, -3.72, -8.01, -7.40],
    'Win Rate': [53.01, 51.53, 51.43, 49.82]
}

df = pd.DataFrame(data)

print('Input Period Statistics:')
print(df)

# --- Aggregating the Statistics ---

# 1. Final Portfolio Value: Multiply the period values to get the overall growth factor
combined_final_value = np.prod(df['Final Portfolio Value'])

# 2. Annualized Sharpe Ratio: (Approximation) Compute the simple average
combined_sharpe = df['Annualized Sharpe'].mean()

# 3. Maximum Drawdown: (Approximation) Use the worst (most negative) drawdown among the periods
combined_drawdown = df['Max Drawdown'].min()

# 4. Win Rate: (Approximation) Average the win rates (assuming equal weights)
combined_win_rate = df['Win Rate'].mean()

# --- Calculate Annualized Return ---
# The combined_final_value represents the cumulative growth factor over 28 days.
trading_days_year = 252
total_period_days = 28  # 4 periods * 7 days each
annualization_factor = trading_days_year / total_period_days
annualized_return = combined_final_value ** annualization_factor - 1

# --- Display the Aggregated Results ---
print('\nCombined Period Statistics:')
print('Combined Final Portfolio Value:', combined_final_value)
print('Combined Annualized Sharpe Ratio:', combined_sharpe)
print('Combined Maximum Drawdown:', combined_drawdown)
print('Combined Win Rate (%):', combined_win_rate)
print('Annualized Return (%):', annualized_return * 100)



Input Period Statistics:
     Period  Final Portfolio Value  Annualized Sharpe  Max Drawdown  Win Rate
0  Period 1                 1.0403             3.8151         -6.77     53.01
1  Period 2                 1.0520             4.6947         -3.72     51.53
2  Period 3                 1.0326             3.1946         -8.01     51.43
3  Period 4                 1.0196             2.1642         -7.40     49.82

Combined Period Statistics:
Combined Final Portfolio Value: 1.1522223253325763
Combined Annualized Sharpe Ratio: 3.46715
Combined Maximum Drawdown: -8.01
Combined Win Rate (%): 51.4475
Annualized Return (%): 257.95346629257716
